In [ ]:
import metpy
import cartopy.crs as ccrs
import hvplot.xarray
import matplotlib.pyplot as plt

Use metpy's `parse_cf()` to generate projection information for future plotting

In [ ]:
# fs2 = fsspec.filesystem('reference', fo="./example_jsons/combined.json", remote_protocol='s3', remote_options=dict(anon=True), skip_instance_cache=True)
fs2 = fsspec.filesystem('reference', fo="./combined.json", remote_protocol='s3', remote_options=dict(anon=True), skip_instance_cache=True)
ds = xr.open_dataset(fs2.get_mapper(""), engine='zarr').metpy.parse_cf()
ds

Use metpy to calculate lat/lon based on the GOES projection grid, and rename time dimension (for better plotting with hvplot)

In [ ]:
ds = ds.metpy.assign_latitude_longitude()
ds['t'].attrs['long_name'] = 'Time'

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
%%time
mask_lat = (ds.latitude > 20) & (ds.latitude < 60)
mask_lon = (ds.longitude > -100) & (ds.longitude < -50)


sst = ds.SST.where(mask_lat & mask_lon, drop=True)

In [ ]:
sst.hvplot.quadmesh(
    'x', 'y', groupby='t',
    crs = sst.metpy.cartopy_crs, projection=ccrs.Orthographic(-75,30),
    features = ['land', 'borders'], width=500,
    project=True, rasterize=True, coastline=True, cmap='jet'
)

In [ ]:
sst.mean(dim='t').hvplot.quadmesh(
    'x', 'y',
    crs = sst.metpy.cartopy_crs, projection=ccrs.Orthographic(-75,30),
    features = ['land', 'borders'], width=500,
    project=True, rasterize=True, coastline=True, cmap='jet'
)